In [1]:
import os
from osgeo import gdal
import numpy as np

In [2]:
infile="grid_1km/grid_250m.tiff"
in_ds = gdal.Open(infile)

In [3]:
in_band = in_ds.GetRasterBand(1)

In [4]:
gtiff_driver = gdal.GetDriverByName('GTiff')
out_ds = gtiff_driver.Create('grid_1km/grid_250m_consecutive.tiff', in_band.XSize, in_band.YSize, 1, in_band.DataType)
out_ds.SetProjection(in_ds.GetProjection())
out_ds.SetGeoTransform(in_ds.GetGeoTransform())

0

In [5]:
in_data = in_band.ReadAsArray()

In [9]:
in_masked = np.ma.masked_where(in_data == 0, in_data)

In [12]:
ndv = 0
in_ones = np.ma.array(np.ones(in_masked.shape), mask=in_masked.mask, fill_value=ndv)

In [17]:
in_ones_normal = np.ma.getdata(in_ones)

In [19]:
in_ones_normal[in_ones.mask] = 0

In [20]:
in_ones_cumulative = np.reshape(np.cumsum(in_ones_normal), in_ones_normal.shape)
in_ones_cumulative[in_ones.mask] = 0

In [21]:
out_band = out_ds.GetRasterBand(1)
out_band.SetNoDataValue(ndv)
out_band.WriteArray(in_ones_cumulative)
out_band.FlushCache()
out_band.ComputeStatistics(False)
del out_ds